In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('Daily Household Transactions.csv')
df.head(10)

,Date,Mode,Category,Subcategory,Note,Amount,Income/Expense,Currency
0,20/09/2018 12:04:08,Cash,Transportation,Train,2 Place 5 to Place 0,30.0,Expense,INR
1,20/09/2018 12:03:15,Cash,Food,snacks,Idli medu Vada mix 2 plates,60.0,Expense,INR
2,19/09/2018,Saving Bank account 1,subscription,Netflix,1 month subscription,199.0,Expense,INR
3,17/09/2018 23:41:17,Saving Bank account 1,subscription,Mobile Service Provider,Data booster pack,19.0,Expense,INR
4,16/09/2018 17:15:08,Cash,Festivals,Ganesh Pujan,Ganesh idol,251.0,Expense,INR
5,15/09/2018 06:34:17,Credit Card,subscription,Tata Sky,Permanent Residence - Tata Play recharge,200.0,Expense,INR
6,14/09/2018 05:39:17,Cash,Transportation,auto,Place 2 station to Permanent Residence,50.0,Expense,INR
7,13/09/2018 21:35:15,Saving Bank account 1,Transportation,Train,2 Place 0 to Place 3,40.0,Expense,INR
8,13/09/2018 21:01:47,Credit Card,Other,NaN,HBR 2 Months subscription,83.0,Expense,INR
9,13/09/2018 21:01:32,Cash,Food,Grocery,1kg atta,46.0,Expense,INR


In [4]:
df['Note'] = df['Note'].str.lower()
df['Subcategory'] = df['Subcategory'].str.lower()
df['Category'] = df['Category'].str.lower()

In [5]:
df = df.loc[df['Category'].str.strip().str.split().str.len() == 1]
df = df.dropna(subset=['Subcategory', 'Note'])

In [6]:
X = df['Subcategory'].astype(str) + ' ' + df['Note']
y = df['Category'].astype(str)
X, y

(0                      train 2 place 5 to place 0
 1              snacks idli medu vada mix 2 plates
 2                    netflix 1 month subscription
 3       mobile service provider data booster pack
 4                        ganesh pujan ganesh idol
                           ...                    
 2329                      train place 0 to bandra
 2330                         footwear shoe polish
 2331                  train to and fro mum to brc
 2333                             health face mask
 2334                              medicine koflet
 Length: 1562, dtype: object,
 0       transportation
 1                 food
 2         subscription
 3         subscription
 4            festivals
              ...      
 2329    transportation
 2330           apparel
 2331    transportation
 2333            health
 2334            health
 Name: Category, Length: 1562, dtype: object)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [68]:
model = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', CatBoostClassifier(iterations=5000, learning_rate=1e-4 * 5, auto_class_weights='Balanced'))
])
model.fit(X_train, y_train)

0:	learn: 2.7078963	total: 43.5ms	remaining: 3m 37s
1:	learn: 2.7073727	total: 94.5ms	remaining: 3m 56s
2:	learn: 2.7071760	total: 137ms	remaining: 3m 47s
3:	learn: 2.7066187	total: 184ms	remaining: 3m 49s
4:	learn: 2.7065085	total: 224ms	remaining: 3m 44s
5:	learn: 2.7064178	total: 263ms	remaining: 3m 38s
6:	learn: 2.7062842	total: 304ms	remaining: 3m 36s
7:	learn: 2.7059371	total: 342ms	remaining: 3m 33s
8:	learn: 2.7056553	total: 379ms	remaining: 3m 30s
9:	learn: 2.7054704	total: 415ms	remaining: 3m 27s
10:	learn: 2.7050990	total: 451ms	remaining: 3m 24s
11:	learn: 2.7046556	total: 487ms	remaining: 3m 22s
12:	learn: 2.7046193	total: 520ms	remaining: 3m 19s
13:	learn: 2.7041569	total: 554ms	remaining: 3m 17s
14:	learn: 2.7040972	total: 589ms	remaining: 3m 15s
15:	learn: 2.7037833	total: 627ms	remaining: 3m 15s
16:	learn: 2.7036175	total: 665ms	remaining: 3m 14s
17:	learn: 2.7031893	total: 708ms	remaining: 3m 16s
18:	learn: 2.7031033	total: 754ms	remaining: 3m 17s
19:	learn: 2.7028528

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x7f79417d6580>)])

In [69]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.731629392971246

In [80]:
X_test, y_pred

(1080                            trip tours and travel
 1371                                 dinner 4 chapati
 1711           grocery milk + butter + biscuits/toast
 1043     stationary 15 fevistick + 12 xerox2 & print2
 1723    flour mill 1 sher bajri 1 kg dadar 1 kg makka
                             ...                      
 2150                         mutual fund  mutual fund
 1138      snacks garlic bread + periperi masala fries
 1762                        vegetables kandyachi paat
 1968                        mutual fund mutual fund b
 1482                              appliances usb cord
 Length: 313, dtype: object,
 array([['tourism'],
        ['food'],
        ['food'],
        ['education'],
        ['food'],
        ['subscription'],
        ['subscription'],
        ['festivals'],
        ['food'],
        ['household'],
        ['food'],
        ['festivals'],
        ['food'],
        ['food'],
        ['food'],
        ['household'],
        ['food'],
        ['transp

In [43]:
y.value_counts()

Category
food              746
transportation    278
household         161
subscription       92
health             87
investment         55
apparel            46
family             23
beauty             19
education          18
festivals          16
culture            10
tourism             5
grooming            4
documents           2
Name: count, dtype: int64

In [90]:
val = model.predict(['fries'])
val

array([['festivals']], dtype=object)

In [67]:
model.named_steps['vectorizer'].get_feature_names_out()

array(['10', '100', '1000', '1017', '10pcs', '11', '110', '12', '120',
       '128gb', '1299', '13', '130', '138', '15', '150', '1500', '16',
       '17', '170', '180', '1800', '1850', '190', '1kg', '1l', '1lit',
       '20', '200', '2024', '210', '220', '2200', '226', '24', '25',
       '250ml', '27', '28', '280', '2gb', '2kg', '30', '30mb', '32', '33',
       '35', '36', '3ac', '3d', '3kg', '3m', '3tier', '40', '40mbps',
       '42', '43', '45meters', '482', '4dx', '4kg', '50', '500gb', '52',
       '540', '54rsx2', '55', '59', '5rs', '5star', '60', '64', '75',
       '7756', '80', '85', '8pcs', '90', '999', 'aadhar', 'aakash', 'ac',
       'accessories', 'acid', 'acne', 'address', 'adulsa', 'adults',
       'advance', 'agarbatti', 'aid', 'airport', 'alfa', 'all', 'aloo',
       'altering', 'amazon', 'amritsar', 'amrutullya', 'amul', 'and',
       'annual', 'antibacterial', 'antivirus', 'ants', 'apple',
       'appliances', 'april', 'aqugurad', 'at', 'atta', 'audible', 'auto',
      